<h1> DS200A Computer Vision Assignment</h1>

<h2>  Part Four: Extension Activities</h2>	


TensorFlow (Optional)- Now, try using TensorFlow to categorize your images. The accuracy should be significantly higher due to the usage of nueral nets



In [1]:
import tensorflow as tf
import keras
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

Using TensorFlow backend.


In [2]:
train_df = pd.read_pickle("train_df.pkl")
train_df = train_df.drop(labels="filename", axis = 1)
train, valid = train_test_split(train_df, test_size=0.2, random_state=42, shuffle=True, stratify=train_df['class'])

In [3]:
train_x, train_y = np.array(train.iloc[:, 1:]), np.array(train['class'])
valid_x, valid_y = np.array(valid.iloc[:, 1:]), np.array(valid['class'])

In [4]:
model = keras.Sequential()
model.add(keras.layers.Dense(128, input_shape = (len(train_df.loc[0, :]) - 1,), activation = tf.nn.relu))
model.add(keras.layers.Dense(64, activation = tf.nn.relu, kernel_regularizer=keras.regularizers.l2(0.001)))
model.add(keras.layers.Dense(32, activation = tf.nn.relu, kernel_regularizer=keras.regularizers.l2(0.001)))
model.add(keras.layers.Dense(20, activation = tf.nn.sigmoid))

model.compile(optimizer=tf.train.AdamOptimizer(), 
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
model.fit(train_x, train_y, batch_size = 128, epochs = 250)

Epoch 1/250
1200/1200 [==============================] - 0s 347us/step - loss: 3.6089 - acc: 0.0300
Epoch 2/250
1200/1200 [==============================] - 0s 39us/step - loss: 3.1088 - acc: 0.0467
Epoch 3/250
1200/1200 [==============================] - 0s 31us/step - loss: 3.0737 - acc: 0.0500
Epoch 4/250
1200/1200 [==============================] - 0s 32us/step - loss: 3.0495 - acc: 0.0408
Epoch 5/250
1200/1200 [==============================] - 0s 50us/step - loss: 3.0269 - acc: 0.0358
Epoch 6/250
1200/1200 [==============================] - 0s 26us/step - loss: 3.0171 - acc: 0.0250
Epoch 7/250
1200/1200 [==============================] - 0s 39us/step - loss: 3.0016 - acc: 0.0250
Epoch 8/250
1200/1200 [==============================] - 0s 32us/step - loss: 2.9922 - acc: 0.0308
Epoch 9/250
1200/1200 [==============================] - 0s 39us/step - loss: 2.9891 - acc: 0.0358
Epoch 10/250
1200/1200 [==============================] - 0s 31us/step - loss: 2.9809 - acc: 0.0508
Epoch 11

1200/1200 [==============================] - 0s 31us/step - loss: 2.2677 - acc: 0.2717
Epoch 83/250
1200/1200 [==============================] - 0s 39us/step - loss: 2.2045 - acc: 0.2850
Epoch 84/250
1200/1200 [==============================] - 0s 31us/step - loss: 2.2148 - acc: 0.2992
Epoch 85/250
1200/1200 [==============================] - 0s 26us/step - loss: 2.2197 - acc: 0.2850
Epoch 86/250
1200/1200 [==============================] - 0s 26us/step - loss: 2.2323 - acc: 0.2900
Epoch 87/250
1200/1200 [==============================] - 0s 31us/step - loss: 2.2444 - acc: 0.2742
Epoch 88/250
1200/1200 [==============================] - 0s 13us/step - loss: 2.1772 - acc: 0.2967
Epoch 89/250
1200/1200 [==============================] - 0s 26us/step - loss: 2.1716 - acc: 0.2867
Epoch 90/250
1200/1200 [==============================] - 0s 26us/step - loss: 2.1885 - acc: 0.2950
Epoch 91/250
1200/1200 [==============================] - 0s 31us/step - loss: 2.2405 - acc: 0.2967
Epoch 92/250


1200/1200 [==============================] - 0s 26us/step - loss: 1.9106 - acc: 0.3817
Epoch 164/250
1200/1200 [==============================] - 0s 39us/step - loss: 1.8829 - acc: 0.3767
Epoch 165/250
1200/1200 [==============================] - 0s 32us/step - loss: 1.8425 - acc: 0.3975
Epoch 166/250
1200/1200 [==============================] - 0s 39us/step - loss: 1.8582 - acc: 0.3908
Epoch 167/250
1200/1200 [==============================] - 0s 45us/step - loss: 1.8892 - acc: 0.3808
Epoch 168/250
1200/1200 [==============================] - 0s 39us/step - loss: 1.8769 - acc: 0.3775
Epoch 169/250
1200/1200 [==============================] - 0s 49us/step - loss: 1.8480 - acc: 0.3967
Epoch 170/250
1200/1200 [==============================] - 0s 41us/step - loss: 1.8569 - acc: 0.3892
Epoch 171/250
1200/1200 [==============================] - 0s 22us/step - loss: 1.8345 - acc: 0.3867
Epoch 172/250
1200/1200 [==============================] - 0s 39us/step - loss: 1.8311 - acc: 0.3908
Epoc

1200/1200 [==============================] - 0s 31us/step - loss: 1.6859 - acc: 0.4417
Epoch 245/250
1200/1200 [==============================] - 0s 39us/step - loss: 1.7142 - acc: 0.4383
Epoch 246/250
1200/1200 [==============================] - 0s 26us/step - loss: 1.6995 - acc: 0.4333
Epoch 247/250
1200/1200 [==============================] - 0s 31us/step - loss: 1.6612 - acc: 0.4533
Epoch 248/250
1200/1200 [==============================] - 0s 39us/step - loss: 1.6804 - acc: 0.4383
Epoch 249/250
1200/1200 [==============================] - 0s 29us/step - loss: 1.6920 - acc: 0.4392
Epoch 250/250
1200/1200 [==============================] - 0s 27us/step - loss: 1.7268 - acc: 0.4267


In [5]:
test_loss, test_acc = model.evaluate(valid_x, valid_y)
print(test_loss, test_acc)


301/301 [==============================] - 0s 104us/step
2.76450187027 0.318936877571
